![logo](https://github.com/CitrineInformatics/community-tools/blob/master/templates/fig/citrine_banner_2.png?raw=true)

# Data Cleaning Workshop

*Authors: Enze Chen, Anirudh Kashyap, Chris Borg, Malcolm Davidson, Zachary del Rosario*

This notebook walks through a data extraction, formatting, and validation exercise where we progressively spot errors in our data and fix them to improve model quality.

### Learning outcomes
By working through this notebook, you will be able to:
* Understand proper data formatting for machine learning (ML).
* Identify common challenges and errors when working with materials data.
* Use standard programming tools to extract, format, and validate data for ML.


## Table of Contents
0. [Imports](#Python-package-imports)
1. [Inspect data](#Step-1:-Load-and-inspect-the-data)
1. [Extract data](#Step-2:-Data-extraction-with-Tabula)
1. [Standardize headers](#Step-3:-Standardize-the-column-headers)
1. [Impute missing values](#Step-4:-Impute-missing-values)
1. [Generate formula](#Step-5:-Generate-a-chemical-formula)
1. [Visualize data](#Step-6:-Visualize-the-data)
1. [Correct data](#Step-7:-Correct-the-data-by-consulting-a-reference/expert)
1. [Upload data](#Step-8:-Upload-the-data-to-Citrination)

## Python package imports
We will handle all the imports up front so errors can be caught right away. Many of the methods below are written and documented in `helper_functions.py`. Please refer to that file if you would like to learn about the implementation details.

In [ ]:
# IPython magic settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Third-party packages
import pandas as pd              # library for working with tabular data
import matplotlib.pyplot as plt  # library for visualizing data
from workshop_utils import *   # Python file we wrote with helper functions


## DATA EXTRACTION

## Step 1: Load and inspect the data

[Back to ToC](#Table-of-Contents)

Whenever you get data, it's always a good idea to look at the contents and structure. Using the [`pandas`](https://pandas.pydata.org/) package, we will:
1. Read in the CSV file with the `read_csv()` function.
2. Store it in a [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).
3. Print out the first $n$ rows as a sanity check with the `head(n=)` method.

Today we will *use* the pandas package, while tomorrow we'll learn it in greater detail.

In [ ]:
# Load and inspect the data
pd.set_option('display.max_columns', 500)  # do not truncate columns

df1 = pd.read_csv(os.path.join('data', 'messy_data.csv'))

# Display the first 5 rows
df1.head(5)


### Q1: Inspect the raw data
Is there anything about the above DataFrame that looks concerning? Make at least two observations about the data above.


---

Below we will cover two useful methods for DataFrame objects.

In [ ]:
# Print out the data type of each column
df1.dtypes

In [ ]:
# Print out basic summary statistics of each column
df1.describe()

Now let's go back and fix the issue with the column headers.

## Step 2: Data extraction with Tabula

[Back to ToC](#Table-of-Contents)

We will use [Tabula](https://tabula.technology/), an open-source tool for table extraction, to collect some more information about the column headers. 

### Q2: Extract the column name data
In this step we will extract a *mapping* from the mysterious column names above to more human-readable names. We will need to follow precise steps to ensure the data are in the correct format for import. Follow the instructions below.

**Follow these Instructions**:
1. Please download and install Tabula if you haven’t done so already. Run it.
1. Browse for and Import the `Agrawal_table_excerpt.pdf` file.
1. Drag a selection over the table and export the result to CSV.
  - Make sure to **exclude** the table name `Table 1 NIMS data features`. The first row in your CSV should be `Abbreviation`, `Details`.
1. Add a row mapping `No.` to `Sample Number`.
  - Make sure to add this row **after** `Abbreviation, Details`.
1. Change mapping for `Fatigue` to be `Fatigue Strength`.
1. Save the CSV file  and move it to the `data` folder in your workshop directory.
  - Tabula will save this as `tabula-Agrawal_table_excerpt.csv` by default; you should not need to rename the file.

## DATA FORMATTING

## Step 3: Standardize the column headers

[Back to ToC](#Table-of-Contents)

Standardizing the column headers will enable:
* Consistent formatting across multiple datasets.
* More descriptive names for better understanding.
* Use of the [Template CSV Ingester](https://help.citrination.com/knowledgebase/articles/1188136-citrine-template-csv-csv) to obtain PIFs (though we will not show this here).

The following Python code will take the table you just extracted and rename the column headers in your CSV file. Implementation details can be found in the [`helper_functions.py`](helper_functions.py) file.

In [ ]:
# Use a helper function to extract the mapping you extracted
header_mapping = create_mapping_from_table(os.path.join('data', 'tabula-Agrawal_table_excerpt.csv'))
header_mapping


Now that we have the dictionary mapping can load the data and call a helper function to rename the columns.

### Q3: Remap the column names
Complete the code below to give the columns human-readable names.

In [ ]:
###
# TASK: Rename the columns
# TODO: Complete the code below with
# rename_columns_in_df(df = ?, mapping = ?)
# Use the mapping we created in the previous step
###

# -- NO NEED TO CHANGE THIS CODE -----
# Read in the current dataset into a DataFrame
df3 = pd.read_csv(os.path.join('data', 'messy_data.csv'))

# -- UNCOMMENT AND COMPLETE THIS CODE -----
# df3 = rename_columns_in_df(df = ?, mapping = ?)


# -- NO NEED TO CHANGE THIS CODE -----
# Save the DataFrame to a new CSV file and show the new column headers
# don't write row index into CSV
df3.to_csv(os.path.join('data', 'messy_data_headers.csv'), index=False)
df3.head(n=5)


What if we just wanted to see the column names, and not the whole DataFrame?

In [ ]:
# Inspect the current column headers
df3.columns


## Step 4: Impute missing values

[Back to ToC](#Table-of-Contents)

The method of imputation of missing values will differ on a case-by-case basis. As explained by Agrawal et al., missing time values we'll assign a value of `0`, which is sensible.

While we normally have to loop through the table and fill in blanks with `0`, you'll notice that pandas imported the blanks as `NaN`, and there is a built-in `fillna(value=)` method for DataFrames that fills `NaN` cells with the value of your choice.

### Q4: Fill missing values
Complete the code below to fill in the missing values.

In [ ]:
###
# TASK: Fill the missing values with zeros
# TODO: Complete the code below with 
# df.fillna(value=?)
###

# -- NO NEED TO CHANGE THIS CODE -----
df4 = pd.read_csv(os.path.join('data', 'messy_data_headers.csv'))

# -- UNCOMMENT AND COMPLETE THIS LINE -----
# df4 = df4.fillna(value=?)


# -- NO NEED TO CHANGE THIS CODE -----
# Write the data with imputed values
df4.to_csv(os.path.join('data', 'messy_data_imputed.csv'), index=False) # don't write row index into CSV
df4.head(n=5)

## Step 5: Generate a chemical formula

[Back to ToC](#Table-of-Contents)

In order to take advantage of the [Magpie](https://www.nature.com/articles/npjcompumats201628) library's featurization, we must have a chemical formula. Therefore, we need to generate a chemical formula and append a new column to the DataFrame.

The astute observer will note that the composition fractions currently *do not* add up to 100%. This is because the base element is iron, and the iron content is assumed to be the remainder of the composition. This means we need to calculate the fraction of iron in the sample and add that as a column before generating the chemical formula.

### Q5: Compute the formulae
Complete the code below to compute the chemical formulas.

In [ ]:
###
# TASK: Compute the formulae
# TODO: Complete the code below with the suggested helper functions
###

# -- NO NEED TO MODIFY THIS CODE -----
df5 = pd.read_csv(os.path.join('data', 'messy_data_imputed.csv'))

# Add a column for iron, the base metal, with: df_new = add_iron_composition(df=?)
# Add a column for the chemical formula with: df_new = add_chemical_formula(df=?)
# -- UNCOMMENT AND COMPLETE THIS CODE -----
# df5 = ???


# -- NO NEED TO MODIFY THIS CODE -----
# Write CSV without row indices
df5.to_csv(os.path.join('data', 'messy_data_formula.csv'), index=False)
df5.head(n=5)


## DATA VALIDATION

## Step 6: Visualize the data

[Back to ToC](#Table-of-Contents)

We will create a [histogram](https://en.wikipedia.org/wiki/Histogram) of the target values to look for suspicious rows in our data. The histogram is one of the [seven basic tools of quality](https://en.wikipedia.org/wiki/Seven_basic_tools_of_quality), and therefore is one of the fundamental tools for inspecting data.

The `matplotlib.pyplot` library (which we imported above with the abbreviation `plt`) has the function `hist(data, bins=)` to plot histograms. We will learn how to use `matplotlib` in tomorrow's exercises.  For now, simply interpret the histogram below.

### Q6: Interpret this histogram
There is at least one kind of error remaining in the data; can you determine what it is, based on the following histogram?

In [ ]:
# Create a histogram of the output 'PROPERTY: Fatigue Strength'
df6 = pd.read_csv(os.path.join('data', 'messy_data_formula.csv'))

plt.rcParams.update({'figure.figsize': (8, 6), 'font.size': 14})
plt.hist(df6['PROPERTY: Fatigue Strength'], bins=30)

# Some plot settings
plt.xlabel('Fatigue Strength')
plt.ylabel('Number of Entries')
plt.show()



We can isolate the 'strange' cases by *filtering* our data; we demonstrate this below.

In [ ]:
# What's fishy? Let's inspect the last few rows of the dataset
df6[df6['PROPERTY: Fatigue Strength'] < 2].head()

## Step 7: Correct the data by consulting a reference/expert

[Back to ToC](#Table-of-Contents)

Now that we have identified some suspicious data, we have to go in and manually fix these entries. The correct values will have to be verified against the original data or expert intuition, or discarded entirely.

Upon verification, we learn that fatigue strength values around 1.0 should really be 1000 times larger to account for the correct units. The following helper function will carry out this fix.

In [ ]:
# Fix cells where the Fatigue Strength is too small.
cleaned_data = 'agrawal_steel_fatigue_dataset.csv'
df7 = pd.read_csv(os.path.join('data', 'messy_data_formula.csv'))
df7 = fix_fatigue_strength(df=df7)

# don't write row index into CSV
df7.to_csv(os.path.join('data', cleaned_data), index=False)
df7[df7['PROPERTY: Fatigue Strength'] < 2].head()


Did this fix the problem? Complete the following task to find out.

### Q7: Double-check the data
Copy the code from **Q6** to plot a histogram for the fatigue strength of `df7`. Interpret this new histogram -- are there any problems remaining? What observations can you make about the distribution of fatigue strength values?

In [ ]:
###
# TASK: Plot a histogram of df7['PROPERTY: Fatigue Strength']
# TODO: Copy and modify the code from Q6
# TODO: List your observations about the data below.
###

# -- WRITE YOUR CODE BELOW -----



**Write your observations about the fatigue data here**

## Step 8: Upload the data to Citrination (OPTIONAL)

[Back to ToC](#Table-of-Contents)

Hooray! Now that we have finished cleaning the data, we can upload it to Citrination using the API. We could also use the [Template CSV Ingester](https://help.citrination.com/knowledgebase/articles/1188136-citrine-template-csv-csv) through the UI, but the method we show below is a bit faster. Yay, coding!

To do so, we will first convert the CSV file into a PIF, which involves the use of the [`pypif`](https://github.com/CitrineInformatics/pypif) package. Though we will not discuss the code in much detail below, we encourage you to look through the accompanying Python file.

In [ ]:
# Read in CSV and create output file path
df8 = pd.read_csv(os.path.join('data', cleaned_data))
outfile = cleaned_data[:-4] + '.json'

# Convert CSV to PIF and save to file
csv_to_pifs(df=df8, fpath=os.path.join('data', outfile))

Once we have the PIF created, we can then use the API to upload the dataset to Citrination. Run the following cell.

In [ ]:
# Specify details for the deployment and dataset
site = 'https://citrination.com'
dataset_name = 'GATW Steel Fatigue Dataset ' + str(uuid4())[:6]
dataset_desc = 'Steel fatigue dataset for the GATW 2019.'

# Create a dataset and upload dataset to public Citrination
api_key = getAPIKey()
client = CitrinationClient(api_key=api_key, site=site)
dataset_id = create_and_upload_data(
    client=client,
    fpath=os.path.join('data', outfile),
    dataset_name=dataset_name,
    dataset_desc=dataset_desc,
    public_flag=False
)
print('Dataset successfully created and uploaded!')
print('Find it at {}/datasets/{}'.format(site, dataset_id))


## Conclusion
This concludes the Data Cleaning Workshop. Now you know some steps you can take when cleaning data for MI. When you have more time, you can go through the accompanying `workshop_utils.py` file to learn about the detailed implementations. We are happy to take any remaining questions you may have.